<a href="https://colab.research.google.com/github/ianfanggis/ac-data-course/blob/main/W1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##基礎建設

In [1]:
import os
from google.colab import drive
import numpy as np
drive.mount('/content/drive')

Mounted at /content/drive


In [75]:
# os.chdir('/content/drive/My Drive/2021_AC_re')
# os.listdir()

In [2]:
import pandas as pd
import gzip
import json

import matplotlib.pyplot as plt
import datetime

import warnings
warnings.filterwarnings('ignore')

In [3]:
def parse(path):
  g = gzip.open(path,'rb')
  for l in g:
    yield json.loads(l)
def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

##Data

In [4]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2021-12-26 12:40:45--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv’

All_Beauty.csv      100%[===================>]  14.78M  8.84MB/s    in 1.7s    

2021-12-26 12:40:47 (8.84 MB/s) - ‘All_Beauty.csv’ saved [15499476/15499476]

--2021-12-26 12:40:47--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz’

meta_All_Beauty.jso 100%[===================>]   9.85M  6.59MB/s    in 1.5s    

202

In [5]:
'''
提供商品的基本資訊
共32892 rows, 19 columns

asin - ID of the product, e.g. 0000031852 ← 商品 ID
title - name of the product
feature - bullet-point format features of the product
description - description of the product
price - price in US dollars (at time of crawl)
imageURL - url of the product image
imageURL - url of the high resolution product image
related - related products (also bought, also viewed, bought together, buy after viewing)
salesRank - sales rank information
brand - brand name
categories - list of categories the product belongs to
tech1 - the first technical detail table of the product
tech2 - the second technical detail table of the product
similar - similar product table
'''
from ssl import match_hostname
metadata = getDF('/content/drive/My Drive/2021_AC_re/meta_All_Beauty.json.gz')
display(metadata.shape)
display(metadata.head(3))

(32892, 19)

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]


In [6]:
'''
提供使用者購買商品的紀錄
評論資料，共371345rows, 4 columns
訓練資料：2000-01-10 - 2018-09-01 (共 370752)
測試資料：2018-09-01 - 2018-09-30 (共 590)

reviewerID - ID of the reviewer, e.g. A2SUAM1J3GNN3B ← 用戶 ID
asin - ID of the product, e.g. 0000013714 ← 商品 ID
overall - rating of the product ← 用戶對商品的評分
unixReviewTime - time of the review (unix time) ← 留下評論的時間戳記

'''
ratings = pd.read_csv('All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)
display(ratings.head())
display(ratings.shape)

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


(371345, 4)

##整理資料

###metadata

In [7]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32892 entries, 0 to 32891
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   category         32892 non-null  object
 1   tech1            32892 non-null  object
 2   description      32892 non-null  object
 3   fit              32892 non-null  object
 4   title            32892 non-null  object
 5   also_buy         32892 non-null  object
 6   tech2            32892 non-null  object
 7   brand            32892 non-null  object
 8   feature          32892 non-null  object
 9   rank             32892 non-null  object
 10  also_view        32892 non-null  object
 11  details          32892 non-null  object
 12  main_cat         32892 non-null  object
 13  similar_item     32892 non-null  object
 14  date             32892 non-null  object
 15  price            32892 non-null  object
 16  asin             32892 non-null  object
 17  imageURL         32892 non-null

In [9]:
'''
檢查空值
'''
metadata_m= metadata.mask(metadata.applymap(str).eq('[]')).replace('', np.nan)
display(metadata_m.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32892 entries, 0 to 32891
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   category         0 non-null      float64
 1   tech1            10 non-null     object 
 2   description      15119 non-null  object 
 3   fit              0 non-null      float64
 4   title            32891 non-null  object 
 5   also_buy         6597 non-null   object 
 6   tech2            0 non-null      float64
 7   brand            17219 non-null  object 
 8   feature          269 non-null    object 
 9   rank             32515 non-null  object 
 10  also_view        8132 non-null   object 
 11  details          32892 non-null  object 
 12  main_cat         32892 non-null  object 
 13  similar_item     1304 non-null   object 
 14  date             19 non-null     object 
 15  price            11459 non-null  object 
 16  asin             32892 non-null  object 
 17  imageURL    

None

In [10]:
'''
檢查是否有重複，404筆資料重複，並移除
'''
display(metadata_m[metadata_m.astype(str).duplicated() == True].shape)

metadata_m = metadata_m.astype(str).drop_duplicates().reset_index(drop = True)
display(metadata_m.head(2))
display(metadata_m.shape)

(404, 19)

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,nan,nan,"[""Loud 'N Clear Personal Sound Amplifier allow...",nan,Loud 'N Clear&trade; Personal Sound Amplifier,nan,nan,idea village,nan,"2,938,573 in Beauty & Personal Care (",nan,{'ASIN: ': '6546546450'},All Beauty,nan,nan,nan,6546546450,nan,nan
1,nan,nan,['No7 Lift & Luminate Triple Action Serum 50ml...,nan,No7 Lift &amp; Luminate Triple Action Serum 50...,"['B01E7LCSL6', 'B008X5RVME']",nan,nan,nan,"872,854 in Beauty & Personal Care (",nan,"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",nan,$44.99,7178680776,nan,nan


(32488, 19)

In [11]:
metadata_m.columns

Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'tech2',
       'brand', 'feature', 'rank', 'also_view', 'details', 'main_cat',
       'similar_item', 'date', 'price', 'asin', 'imageURL', 'imageURLHighRes'],
      dtype='object')

In [12]:
'''
保留需要的資訊
'''

metadata_f = metadata_m.loc[:,['asin', 'rank', 'brand', 'price', 'also_view', 'also_buy','similar_item']]
metadata_f.head(2)

,asin,rank,brand,price,also_view,also_buy,similar_item
0,6546546450,"2,938,573 in Beauty & Personal Care (",idea village,nan,nan,nan,nan
1,7178680776,"872,854 in Beauty & Personal Care (",nan,$44.99,nan,"['B01E7LCSL6', 'B008X5RVME']","class=""a-bordered a-horizontal-stripes a-spa..."


In [13]:
'''
移除price特殊符號
'''
metadata_f['price'] = metadata_f['price'].str.replace(r'[$]','',regex = True)

In [14]:
'''
rank 切分
'''
metadata_f['rank_1'] = metadata_f['rank'].apply(lambda x: x.split('in')[0])


In [15]:
metadata_f['sub_ca'] = metadata_f['rank'].apply(lambda x: x.split('in')[-1])

In [16]:
metadata_f = metadata_f.drop('rank',axis = 1)

In [17]:
metadata_f['rank_1'] = metadata_f['rank_1'].str.replace(r',','')

In [18]:
metadata_f['sub_ca'] = metadata_f['sub_ca'].str.replace(r'\(','')
metadata_f['sub_ca']

0         Beauty & Personal Care 
1         Beauty & Personal Care 
2         Beauty & Personal Care 
3         Beauty & Personal Care 
4         Beauty & Personal Care 
                   ...           
32483     Beauty & Personal Care 
32484     Beauty & Personal Care 
32485     Beauty & Personal Care 
32486     Beauty & Personal Care 
32487     Beauty & Personal Care 
Name: sub_ca, Length: 32488, dtype: object

In [19]:
metadata_f['sub_ca'].unique()

array([' Beauty & Personal Care ', ' Beauty &amp; Personal Care ', 'nan',
       ' Grocery & Gourmet Food ',
       " Beauty & Personal Care See top 100)']",
       ' Grocery &amp; Gourmet Food ', ' Health &amp; Household ',
       " Beauty & Personal Care)']", " Grocery & Gourmet Food)']",
       ' Sports & Outdoors ', ' Health & Household ',
       " Beauty & Personal Care > Fragrance > Aromatherapy']",
       ' Toys & Games ', ' Baby ', 'g, Shoes & Jewelry ',
       ' Home & Kitchen ', ' Automotive ', ' Tools & Home Improvement ',
       " Beauty & Personal Care > Gift Sets']"], dtype=object)

In [20]:
metadata_f.head()

,asin,brand,price,also_view,also_buy,similar_item,rank_1,sub_ca
0,6546546450,idea village,nan,nan,nan,nan,2938573,Beauty & Personal Care
1,7178680776,nan,44.99,nan,"['B01E7LCSL6', 'B008X5RVME']","class=""a-bordered a-horizontal-stripes a-spa...",872854,Beauty & Personal Care
2,7250468162,No7,28.76,"['B01B8BR0O8', 'B01B8BR0NO', 'B014MHXXM8']",nan,nan,956696,Beauty & Personal Care
3,7367905066,nan,nan,nan,['B0041PBXX8'],nan,1870258,Beauty & Personal Care
4,7414204790,Pirmal Healthcare,12.15,"['3254895630', 'B007VL1D9S', 'B00EH9A0RI', 'B0...",nan,nan,67701,Beauty & Personal Care


In [21]:
# 十位數轉時間指標
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'],unit = 's')

##資料切分

In [96]:
ratings_trainings = ratings[(ratings['DATE'] < '2018-09-01')]

In [97]:
display(ratings_trainings.shape)
display(ratings_trainings.head())

(370752, 5)

,asin,reviewerID,overall,unixReviewTime,DATE
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000,2015-02-19
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800,2014-12-18
2,0143026860,A1572GUYS7DGSR,4.0,1407628800,2014-08-10
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000,2013-03-11
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200,2011-12-25


In [104]:
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())
users

['A100XQFWKQ30O2',
 'A103T1QOGFCSEH',
 'A106UKKSJ2KXPF',
 'A10A7GV4D5A11V',
 'A1119JJ37ZLB8R',
 'A113UOOLBSZN52',
 'A12M4U7WK4ALCR',
 'A12T8YTW6VWT7S',
 'A1364JXGKB46MM',
 'A137DALOQFKBTI',
 'A13FEZ3WV7S2EY',
 'A13IV4I1B0RXMG',
 'A13JU88JAHN72I',
 'A13K55R6VH1OOD',
 'A13P7VFU075A',
 'A13SWYE4QLB6NG',
 'A13ZTQ0Q4ATA41',
 'A142EDN04OD62U',
 'A142I22FIC8MZK',
 'A14834QTII5TLT',
 'A14A447VPACTBC',
 'A14AP6MN5XO6LB',
 'A14CLF25IX25US',
 'A14LYXC3HTBAHI',
 'A14VUW4KZ34EOE',
 'A14Y32P26G9YL',
 'A157T25PBS7MX4',
 'A15HZDSERD85C8',
 'A15JJ8J1FGADIX',
 'A15ZCL70JXXH89',
 'A1617KN2IAWZ6J',
 'A16E0O88262HKA',
 'A16NSZ58PTVIYF',
 'A16UGDXRTDLJG5',
 'A16X9HR3UFQQXY',
 'A16Y7V1CZCWKFV',
 'A174YOBOSW9WDN',
 'A1786SKRAJXH86',
 'A17K2BUZ20WD2',
 'A17LYRFV645L0V',
 'A18LNGVXDZBTUR',
 'A19503XX7GU6J2',
 'A19HVHRZDYFEOP',
 'A19JM38B861BO4',
 'A19KGY3W1P0MHT',
 'A19KJPOF3QOJ24',
 'A19U5RT4Q8Q1T6',
 'A1AQNQUNX8B6JT',
 'A1ARU0IN6RUFFD',
 'A1BB4635KSKDMV',
 'A1BCC7X6KRTBG0',
 'A1BPJ56BXENC7',
 'A1C0RI6W65FC8B'

In [105]:
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user

[{'asin': ['B01DKQAXC0'], 'reviewerID': 'A100XQFWKQ30O2'},
 {'asin': ['B01DKQAXC0'], 'reviewerID': 'A103T1QOGFCSEH'},
 {'asin': ['B01ENS6XDC'], 'reviewerID': 'A106UKKSJ2KXPF'},
 {'asin': ['B01BFSNF66'], 'reviewerID': 'A10A7GV4D5A11V'},
 {'asin': ['B01CJNZKZK'], 'reviewerID': 'A1119JJ37ZLB8R'},
 {'asin': ['B01EKY0M22'], 'reviewerID': 'A113UOOLBSZN52'},
 {'asin': ['B01DKQAXC0'], 'reviewerID': 'A12M4U7WK4ALCR'},
 {'asin': ['B01G53GFKO'], 'reviewerID': 'A12T8YTW6VWT7S'},
 {'asin': ['B01GLA54SA'], 'reviewerID': 'A1364JXGKB46MM'},
 {'asin': ['B01AWXGD3M'], 'reviewerID': 'A137DALOQFKBTI'},
 {'asin': ['B01BHN3EHE'], 'reviewerID': 'A13FEZ3WV7S2EY'},
 {'asin': ['B01DKQAXC0'], 'reviewerID': 'A13IV4I1B0RXMG'},
 {'asin': ['B017I6B6GK'], 'reviewerID': 'A13JU88JAHN72I'},
 {'asin': ['B01FNJ9MOW'], 'reviewerID': 'A13K55R6VH1OOD'},
 {'asin': ['B01H3ZQ2NI'], 'reviewerID': 'A13P7VFU075A'},
 {'asin': ['B01DVLHVPQ'], 'reviewerID': 'A13SWYE4QLB6NG'},
 {'asin': ['B01AE1TJV0'], 'reviewerID': 'A13ZTQ0Q4ATA41'},

In [106]:
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }

In [107]:
ratings_testings_by_user

{'A100XQFWKQ30O2': ['B01DKQAXC0'],
 'A103T1QOGFCSEH': ['B01DKQAXC0'],
 'A106UKKSJ2KXPF': ['B01ENS6XDC'],
 'A10A7GV4D5A11V': ['B01BFSNF66'],
 'A1119JJ37ZLB8R': ['B01CJNZKZK'],
 'A113UOOLBSZN52': ['B01EKY0M22'],
 'A12M4U7WK4ALCR': ['B01DKQAXC0'],
 'A12T8YTW6VWT7S': ['B01G53GFKO'],
 'A1364JXGKB46MM': ['B01GLA54SA'],
 'A137DALOQFKBTI': ['B01AWXGD3M'],
 'A13FEZ3WV7S2EY': ['B01BHN3EHE'],
 'A13IV4I1B0RXMG': ['B01DKQAXC0'],
 'A13JU88JAHN72I': ['B017I6B6GK'],
 'A13K55R6VH1OOD': ['B01FNJ9MOW'],
 'A13P7VFU075A': ['B01H3ZQ2NI'],
 'A13SWYE4QLB6NG': ['B01DVLHVPQ'],
 'A13ZTQ0Q4ATA41': ['B01AE1TJV0'],
 'A142EDN04OD62U': ['B01CW24JXC'],
 'A142I22FIC8MZK': ['B01CD7JK9E'],
 'A14834QTII5TLT': ['B01GK8P0CS'],
 'A14A447VPACTBC': ['B01B3R5EFO'],
 'A14AP6MN5XO6LB': ['B01E7UKR38'],
 'A14CLF25IX25US': ['B01DLR9IDI'],
 'A14LYXC3HTBAHI': ['B01FT6TMQM'],
 'A14VUW4KZ34EOE': ['B01DKQAXC0'],
 'A14Y32P26G9YL': ['B018WCT01C'],
 'A157T25PBS7MX4': ['B01CX5KCIE'],
 'A15HZDSERD85C8': ['B01DDWA5II'],
 'A15JJ8J1FGADIX': ['B0

In [108]:
users = list(ratings_testings_by_user.keys())
users

['A100XQFWKQ30O2',
 'A103T1QOGFCSEH',
 'A106UKKSJ2KXPF',
 'A10A7GV4D5A11V',
 'A1119JJ37ZLB8R',
 'A113UOOLBSZN52',
 'A12M4U7WK4ALCR',
 'A12T8YTW6VWT7S',
 'A1364JXGKB46MM',
 'A137DALOQFKBTI',
 'A13FEZ3WV7S2EY',
 'A13IV4I1B0RXMG',
 'A13JU88JAHN72I',
 'A13K55R6VH1OOD',
 'A13P7VFU075A',
 'A13SWYE4QLB6NG',
 'A13ZTQ0Q4ATA41',
 'A142EDN04OD62U',
 'A142I22FIC8MZK',
 'A14834QTII5TLT',
 'A14A447VPACTBC',
 'A14AP6MN5XO6LB',
 'A14CLF25IX25US',
 'A14LYXC3HTBAHI',
 'A14VUW4KZ34EOE',
 'A14Y32P26G9YL',
 'A157T25PBS7MX4',
 'A15HZDSERD85C8',
 'A15JJ8J1FGADIX',
 'A15ZCL70JXXH89',
 'A1617KN2IAWZ6J',
 'A16E0O88262HKA',
 'A16NSZ58PTVIYF',
 'A16UGDXRTDLJG5',
 'A16X9HR3UFQQXY',
 'A16Y7V1CZCWKFV',
 'A174YOBOSW9WDN',
 'A1786SKRAJXH86',
 'A17K2BUZ20WD2',
 'A17LYRFV645L0V',
 'A18LNGVXDZBTUR',
 'A19503XX7GU6J2',
 'A19HVHRZDYFEOP',
 'A19JM38B861BO4',
 'A19KGY3W1P0MHT',
 'A19KJPOF3QOJ24',
 'A19U5RT4Q8Q1T6',
 'A1AQNQUNX8B6JT',
 'A1ARU0IN6RUFFD',
 'A1BB4635KSKDMV',
 'A1BCC7X6KRTBG0',
 'A1BPJ56BXENC7',
 'A1C0RI6W65FC8B'

In [109]:
ratings_trainings.shape

(370752, 5)

###額外指標

##產生推薦結果


In [111]:
'''
以商品的出現次數
'''
def recommender_1(training_data, users=[], k=10):
  '''
  * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
  * users: [] 需要被推薦的使用者
  * k: int 每個使用者需要推薦的商品數
  * recommendations: dict
    {
        使用者一： [推薦商品一, 推薦商品二, ...],
        使用者二： [...], ...
    }
  '''
  recommendations = {}
  '''
  Your Code
  '''

  # df1 = training_data.drop_duplicates(subset='asin', keep='first')
  # popularity = training_data['asin'].value_counts().index.to_list()[0:k]
  df1 = training_data.groupby(by = 'asin').agg({'overall':['mean','size']})['overall'].reset_index()
  popularity = df1.sort_values(by = 'size', ascending=(False))['asin'].tolist()[0:k]
  recommendations = {x: popularity for x in users}

  return recommendations
ratings_by_user_1 = recommender_1(ratings_trainings, users)
ratings_by_user_1

{'A100XQFWKQ30O2': ['B000FOI48G',
  'B000GLRREU',
  '1620213982',
  'B001QY8QXM',
  'B01DKQAXC0',
  'B00W259T7G',
  'B006IB5T4W',
  'B00005JS5C',
  'B0012Y0ZG2',
  'B000WYJTZG'],
 'A103T1QOGFCSEH': ['B000FOI48G',
  'B000GLRREU',
  '1620213982',
  'B001QY8QXM',
  'B01DKQAXC0',
  'B00W259T7G',
  'B006IB5T4W',
  'B00005JS5C',
  'B0012Y0ZG2',
  'B000WYJTZG'],
 'A106UKKSJ2KXPF': ['B000FOI48G',
  'B000GLRREU',
  '1620213982',
  'B001QY8QXM',
  'B01DKQAXC0',
  'B00W259T7G',
  'B006IB5T4W',
  'B00005JS5C',
  'B0012Y0ZG2',
  'B000WYJTZG'],
 'A10A7GV4D5A11V': ['B000FOI48G',
  'B000GLRREU',
  '1620213982',
  'B001QY8QXM',
  'B01DKQAXC0',
  'B00W259T7G',
  'B006IB5T4W',
  'B00005JS5C',
  'B0012Y0ZG2',
  'B000WYJTZG'],
 'A1119JJ37ZLB8R': ['B000FOI48G',
  'B000GLRREU',
  '1620213982',
  'B001QY8QXM',
  'B01DKQAXC0',
  'B00W259T7G',
  'B006IB5T4W',
  'B00005JS5C',
  'B0012Y0ZG2',
  'B000WYJTZG'],
 'A113UOOLBSZN52': ['B000FOI48G',
  'B000GLRREU',
  '1620213982',
  'B001QY8QXM',
  'B01DKQAXC0',
  'B00W

In [115]:
'''
以overall的平均
'''
def recommender_2(training_data, users=[], k=10):
  '''
  * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
  * users: [] 需要被推薦的使用者
  * k: int 每個使用者需要推薦的商品數
  * recommendations: dict
    {
        使用者一： [推薦商品一, 推薦商品二, ...],
        使用者二： [...], ...
    }
  '''
  recommendations = {}
  '''
  Your Code
  '''

  # df1 = training_data.drop_duplicates(subset='asin', keep='first')
  df1 = training_data.groupby(by = 'asin').agg(reviewer_count=("asin", 'count'),avg_rating=("overall", 'mean')).reset_index()
  popularity = df1.sort_values(by = 'avg_rating', ascending=(False))['asin'].tolist()[0:k]
  recommendations = {x: popularity for x in users}

  return recommendations
ratings_by_user_2 = recommender_2(ratings_trainings, users)
# ratings_by_user_2

##結果評估

In [92]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
  '''
  * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
  * ratings_by_user: dict 利用訓練資料學習的推薦商品
  * method: str
  * score: float
  '''
  total = 0
  for d in ratings_testings_by_user:
      if d in ratings_by_user:
          total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))
  score = total / len(ratings_testings)
  return score
# evaluate(ratings_testings_by_user, ratings_by_user)

In [113]:
method_1=evaluate(ratings_testings_by_user, ratings_by_user_1)
method_1

0.08305084745762711

In [116]:
method_2=evaluate(ratings_testings_by_user, ratings_by_user_2)
method_2

0.0